In [ ]:
import nltk

nltk.download('punkt')
import pandas as pd

In [ ]:
import os

df = pd.read_csv(os.path.join('data.csv'), names=['label', 'Title', 'Description'])
df['text'] = (df['Title'] + '. ' + df['Description'])
df.drop(columns=['Title', 'Description'], axis=1, inplace=True)
print(df.head())
print(df['text'][1])

In [ ]:
df['label']

In [ ]:
import re

def split_to_sent(text):
    sentences = re.split(
        r"(((?<!\w\.\w.)(?<!\s\w\.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s(?=[A-Z]))|((?<![\,\-\:])\n(?=[A-Z]|\" )))", text)[
                ::4]
    return sentences

In [ ]:
def split_to_words(sentence):
    words = re.findall(r"\w+@\w+\.\w+|\+\d{1,3}-\d{3}-\d{3}-\d{2}-\d{2}|\w+", sentence)
    return words

In [ ]:
from nltk import WordNetLemmatizer
from nltk import SnowballStemmer
import nltk
nltk.download('wordnet')


def process_file():
    wnl = WordNetLemmatizer()
    sst = SnowballStemmer("english")
    counter = 0
    for index, row in df.iterrows():
        sentences = split_to_sent(row['text'])
        words_dic = []
        counter += 1
        for s in sentences:
            words_dic += split_to_words(s)
            words_dic.append("\n")
        lemmatized = []
        stemmed = []
        original = []
        for w in words_dic:
            w_processed = re.sub(r"[.!?,]$", "", w).lower() # убрать удаление точек и etc. отделить от токена и сохранить отдлеьно (1 балл доделать)
            lemmatized.append(wnl.lemmatize(w_processed))
            stemmed.append(sst.stem(w_processed))
            original.append(w_processed)
        save_to_file(original, lemmatized, stemmed, os.path.join(".", "assets", "annotated-corpus", "train", str(row['label']), str(counter) + ".tsv"))

In [ ]:
def save_to_file(original, lemmatized, stemmed, id):
    with open(id, "w") as f:
        for i in range(len(original)):
            if original[i] == "\n":
                 print("", file=f)
            else:
                print(original[i], stemmed[i], lemmatized[i], sep="\t", file=f)

In [ ]:
process_file()